In [26]:
#import libraries
import requests
import pandas as pd

from bs4 import BeautifulSoup

# Specify the source for weather in Fairfax for seven days
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=38.84185000000008&lon=-77.30904999999996#.YexXy_7MIaZ")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())
len(forecast_items)

<div class="tombstone-container">
 <p class="period-name">
  This
  <br/>
  Afternoon
 </p>
 <p>
  <img alt="This Afternoon: Sunny, with a high near 30. Light west wind. " class="forecast-icon" src="newimages/medium/few.png" title="This Afternoon: Sunny, with a high near 30. Light west wind. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 30 °F
 </p>
</div>


9

In [28]:
# Extracting information from the page

period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

img = tonight.find("img")
desc = img['title']
print(desc)

ThisAfternoon
Sunny
High: 30 °F
This Afternoon: Sunny, with a high near 30. Light west wind. 


In [30]:
# extracting and specifying the features for our dataset
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]

short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Sunny', 'Partly Cloudy', 'Mostly Cloudy', 'Isolated SnowShowers', 'Mostly Sunny', 'Mostly Cloudy', 'ChanceRain/Snow', 'Mostly Cloudy', 'Mostly Sunny']
['High: 30 °F', 'Low: 20 °F', 'High: 37 °F', 'Low: 21 °F', 'High: 36 °F', 'Low: 29 °F', 'High: 43 °F', 'Low: 23 °F', 'High: 31 °F']
['This Afternoon: Sunny, with a high near 30. Light west wind. ', 'Tonight: Partly cloudy, with a low around 20. Calm wind becoming south around 5 mph. ', 'Sunday: Mostly cloudy, with a high near 37. Calm wind becoming south around 6 mph in the afternoon. ', 'Sunday Night: Isolated snow showers before 1am.  Mostly cloudy, then gradually becoming mostly clear, with a low around 21. Calm wind becoming northwest around 6 mph after midnight.  Chance of precipitation is 20%.', 'Monday: Mostly sunny, with a high near 36. Light and variable wind becoming south around 6 mph in the afternoon. ', 'Monday Night: Mostly cloudy, with a low around 29.', 'Tuesday: A chance of rain and snow showers before 1pm, then a chan

In [31]:
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,ThisAfternoon,Sunny,High: 30 °F,"This Afternoon: Sunny, with a high near 30. Li..."
1,Tonight,Partly Cloudy,Low: 20 °F,"Tonight: Partly cloudy, with a low around 20. ..."
2,Sunday,Mostly Cloudy,High: 37 °F,"Sunday: Mostly cloudy, with a high near 37. Ca..."
3,SundayNight,Isolated SnowShowers,Low: 21 °F,Sunday Night: Isolated snow showers before 1am...
4,Monday,Mostly Sunny,High: 36 °F,"Monday: Mostly sunny, with a high near 36. Lig..."
5,MondayNight,Mostly Cloudy,Low: 29 °F,"Monday Night: Mostly cloudy, with a low around..."
6,Tuesday,ChanceRain/Snow,High: 43 °F,Tuesday: A chance of rain and snow showers bef...
7,TuesdayNight,Mostly Cloudy,Low: 23 °F,"Tuesday Night: Mostly cloudy, with a low aroun..."
8,Wednesday,Mostly Sunny,High: 31 °F,"Wednesday: Mostly sunny, with a high near 31."


In [35]:
# average of tempreature for next seven days.

temp_nums = weather["temp"].str.extract('([0-9]+)')
weather["temp_num"] = temp_nums.astype('int')
print(weather["temp_num"][2:9])
weather["temp_num"][2:9].mean()

2    37
3    21
4    36
5    29
6    43
7    23
8    31
Name: temp_num, dtype: int32


31.428571428571427

In [36]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool